<a href="https://colab.research.google.com/github/ykamen/CS4342/blob/main/CS4342_HW5_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
from google.colab import files
import scipy.optimize

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
tr_labels = np.load('drive/MyDrive/fashion_mnist_train_labels.npy', 'r')
tr_images = np.load('drive/MyDrive/fashion_mnist_train_images.npy', 'r').T / 255.0
te_images = np.load('drive/MyDrive/fashion_mnist_test_images.npy', 'r').T / 255.0
te_labels = np.load('drive/MyDrive/fashion_mnist_test_labels.npy', 'r')

n_values = np.max(tr_labels) + 1
tr_labels = np.eye(n_values)[tr_labels]

n_values = np.max(te_labels) + 1
te_labels = np.eye(n_values)[te_labels]

In [173]:
NUM_INPUT = 784  # Number of input neurons
NUM_HIDDEN = 40  # Number of hidden neurons
NUM_OUTPUT = 10  # Number of output neurons
NUM_CHECK = 5  # Number of examples on which to check the gradient

# Given a vector w containing all the weights and biased vectors, extract
# and return the individual weights and biases W1, b1, W2, b2.
# This is useful for performing a gradient check with check_grad.
def unpack (w):
    # Unpack arguments
    start = 0
    end = NUM_HIDDEN*NUM_INPUT
    W1 = w[0:end]
    start = end
    end = end + NUM_HIDDEN
    b1 = w[start:end]
    start = end
    end = end + NUM_OUTPUT*NUM_HIDDEN
    W2 = w[start:end]
    start = end
    end = end + NUM_OUTPUT
    b2 = w[start:end]
    # Convert from vectors into matrices
    W1 = W1.reshape(NUM_HIDDEN, NUM_INPUT)
    W2 = W2.reshape(NUM_OUTPUT, NUM_HIDDEN)
    return W1,b1,W2,b2

# Given individual weights and biases W1, b1, W2, b2, concatenate them and
# return a vector w containing all of them.
# This is useful for performing a gradient check with check_grad.
def pack (W1, b1, W2, b2):
    return np.hstack((W1.flatten(), b1.flatten(), W2.flatten(), b2.flatten()))

def relu(x):
    return np.maximum(0, x)

def softmax(Z):
  yhat = np.exp(Z)
  for i in range(Z.shape[0]):
    temp = np.sum(yhat[i],axis=0)
    yhat[i] = yhat[i]/temp
  return yhat.T

def pc(yhat, y):
  c = 0
  for i in range(y.shape[0]):
    if np.argmax(yhat[i]) == np.argmax(y[i]):
      c = c+1
  return c/y.shape[0]

# Given training images X, associated labels Y, and a vector of combined weights
# and bias terms w, compute and return the cross-entropy (CE) loss, accuracy,
# as well as the intermediate values of the NN.
def fCE (X, Y, w, a=0.01):
    W1, b1, W2, b2 = unpack(w)
    z1 = W1.dot(X).T + np.tile(b1,(X.shape[1],1))
    h1 = relu(z1)
    z2 = W2.dot(h1.T).T + np.tile(b2,(X.shape[1],1))
    yhat = softmax(z2)
    L2 = (1/2)*np.sum(np.square(W2))+(1/2)*np.sum(np.square(W1))
    cost = -np.sum(Y.T*np.log(yhat)/(yhat.shape[1]))+a*L2
    acc = pc(yhat.T,Y)
    return cost, acc, X, z1, h1, W1, W2, yhat


def reluDerivative(x):
  x[x<=0] = 0
  x[x>0] = 1
  return x

# Given training images X, associated labels Y, and a vector of combined weights
# and bias terms w, compute and return the gradient of fCE. You might
# want to extend this function to return multiple arguments (in which case you
# will also need to modify slightly the gradient check code below).
def gradCE (X, Y, w, a=0.01):
    W1, b1, W2, b2 = unpack(w)
    cost, acc, X, z1, h1, W1, W2, yhat = fCE(X,Y,w)
    temp1 = yhat.T-Y
    temp2 = temp1.dot(W2)
    g = np.multiply(temp2,reluDerivative(z1)).T
    grad_b1 = np.mean(g,axis=1)
    grad_b2 = np.mean(yhat.T-Y,axis=0)
    grad_W1 = g.dot(X.T)/X.shape[1] + a*W1
    grad_W2 = (yhat.T - Y).T.dot(h1)/X.shape[1] + a*W2
    return pack(grad_W1, grad_b1, grad_W2, grad_b2)

# Given training and testing datasets and an initial set of weights/biases b,
# train the NN.
def train (trainX, trainY, w, epsilon, alpha, num):
  indeces = np.arange(trainX.shape[1])
  a = int(trainX.shape[1]/num)
  ind = np.split(indeces,a)
  for epoch in range(10):
    for i in range(a):
      gW1,gb1,gW2,gb2 = unpack(gradCE(trainX[:,ind[i]],trainY[ind[i]],w,alpha))
      W1,b1,W2,b2 = unpack(w)
      W1 = W1-epsilon*gW1
      b1 = b1-epsilon*gb1
      W2 = W2-epsilon*gW2
      b2 = b2-epsilon*gb2
      w = pack(W1,b1,W2,b2)
  cost, acc, X, z1, h1, W1, W2, yhat = fCE(trainX, trainY, w, alpha)
  return acc

# identical train, but contains the print statement required for this HW.
# I was too lazy to separate all of the parts after already finishing the code
def train2 (trainX, trainY, w, epsilon, alpha, num):
  print("For testing number of epochs is increased to (=30). Number of hidden neurons is still (=40).")
  indeces = np.arange(trainX.shape[1])
  a = int(trainX.shape[1]/num)
  ind = np.split(indeces,a)
  for epoch in range(30):
    for i in range(a):
      gW1,gb1,gW2,gb2 = unpack(gradCE(trainX[:,ind[i]],trainY[ind[i]],w,alpha))
      W1,b1,W2,b2 = unpack(w)
      W1 = W1-epsilon*gW1
      b1 = b1-epsilon*gb1
      W2 = W2-epsilon*gW2
      b2 = b2-epsilon*gb2
      w = pack(W1,b1,W2,b2)
      if (epoch == 29 and i >= a-20):
        cost, acc, X, z1, h1, W1, W2, yhat = fCE(trainX, trainY, w, alpha)
        print(f"For epoch {epoch+1}, batch #{i+1}. Training cost is {cost} and training accuraccy is {acc}.")
  cost, acc, X, z1, h1, W1, W2, yhat = fCE(trainX, trainY, w, alpha)
  return w

def findBestHyperparameters (trainX, trainY):
    a = int(trainX.shape[1]*.8)
    indeces = np.arange(trainX.shape[1])
    np.random.shuffle(indeces)
    vl = indeces[a:]
    #learning rate = eps, minibatch size = num, regularization strength tested = alpha
    pc_best = 0
    b_alpha = 0
    b_eps = 0
    b_num = 0
    print("Static parameters are number of epochs (=10) and number of hidden neurons (=40).")
    for eps in [.01,.005]:
      for alpha in [.01,.005,.001]:
        for num in [16,32]:
          W1 = 2*(np.random.random(size=(NUM_HIDDEN, NUM_INPUT))/NUM_INPUT**0.5) - 1./NUM_INPUT**0.5
          b1 = 0.01 * np.ones(NUM_HIDDEN)
          W2 = 2*(np.random.random(size=(NUM_OUTPUT, NUM_HIDDEN))/NUM_HIDDEN**0.5) - 1./NUM_HIDDEN**0.5
          b2 = 0.01 * np.ones(NUM_OUTPUT)
          w1 = pack(W1, b1, W2, b2)
          pc = train(trainX[:,vl],trainY[vl],w1,eps,alpha,num)
          print(f"Current best accuraccy is: {pc_best}. For epsilon: {eps}, alpha: {alpha}, and batch size: {num}, total accuraccy is: {pc}.")
          if(pc > pc_best):
            b_alpha = alpha
            b_eps = eps
            b_num = num
            pc_best = pc
    return pc_best, b_eps, b_alpha, b_num

def test(testX, testY, w, alpha):
  cost, acc, X, z1, h1, W1, W2, yhat = fCE(testX, testY, w, alpha)
  print(f"Final accuraccy on the test data set, using optimal hyperparameters, is {acc} and the cross-entropy loss is {cost}")
  return cost, acc

In [169]:
  W1 = 2*(np.random.random(size=(NUM_HIDDEN, NUM_INPUT))/NUM_INPUT**0.5) - 1./NUM_INPUT**0.5
  b1 = 0.01 * np.ones(NUM_HIDDEN)
  W2 = 2*(np.random.random(size=(NUM_OUTPUT, NUM_HIDDEN))/NUM_HIDDEN**0.5) - 1./NUM_HIDDEN**0.5
  b2 = 0.01 * np.ones(NUM_OUTPUT)
  w1 = pack(W1, b1, W2, b2)

In [170]:
trainX,trainY,testX,testY = tr_images,tr_labels,te_images,te_labels

In [171]:
# Check that the gradient is correct on just a few examples (randomly drawn).    
idxs = np.random.permutation(trainX.shape[0])[0:NUM_CHECK]
print("Numerical gradient:")
print(scipy.optimize.approx_fprime(w1, lambda w_: fCE(np.atleast_2d(trainX[:,idxs]), np.atleast_2d(trainY[idxs]), w_)[0], 1e-10))
print("Analytical gradient:")
print(gradCE(np.atleast_2d(trainX[:,idxs]), np.atleast_2d(trainY[idxs]), w1))
print("Discrepancy:")
print(scipy.optimize.check_grad(lambda w_: fCE(np.atleast_2d(trainX[:,idxs]), np.atleast_2d(trainY[idxs]), w_)[0], \
                                lambda w_: gradCE(np.atleast_2d(trainX[:,idxs]), np.atleast_2d(trainY[idxs]), w_), \
                                w1))

Numerical gradient:
[ 0.         -0.00015543 -0.00020428 ...  0.08711254  0.10455636
  0.08867129]
Analytical gradient:
[-2.37472291e-06 -1.58756714e-04 -2.07357609e-04 ...  8.71152388e-02
  1.04557879e-01  8.86741698e-02]
Discrepancy:
2.958584840348561e-06


In [174]:
pc_best, b_eps, b_alpha, b_num = findBestHyperparameters (trainX, trainY)

Static parameters are number of epochs (=10) and number of hidden neurons (=40).
Current best accuraccy is: 0. For epsilon: 0.01, alpha: 0.01, and batch size: 16, total accuraccy is: 0.8339166666666666.
Current best accuraccy is: 0.8339166666666666. For epsilon: 0.01, alpha: 0.01, and batch size: 32, total accuraccy is: 0.8210833333333334.
Current best accuraccy is: 0.8339166666666666. For epsilon: 0.01, alpha: 0.005, and batch size: 16, total accuraccy is: 0.8385833333333333.
Current best accuraccy is: 0.8385833333333333. For epsilon: 0.01, alpha: 0.005, and batch size: 32, total accuraccy is: 0.8226666666666667.
Current best accuraccy is: 0.8385833333333333. For epsilon: 0.01, alpha: 0.001, and batch size: 16, total accuraccy is: 0.8429166666666666.
Current best accuraccy is: 0.8429166666666666. For epsilon: 0.01, alpha: 0.001, and batch size: 32, total accuraccy is: 0.8234166666666667.
Current best accuraccy is: 0.8429166666666666. For epsilon: 0.005, alpha: 0.01, and batch size: 16

In [175]:
test(testX, testY, train2(trainX, trainY, w1, b_eps, b_alpha, b_num), b_alpha)

For testing number of epochs is increased to (=30). Number of hidden neurons is still (=40).
For epoch 30, batch #3731. Training cost is 0.38188530048355035 and training accuraccy is 0.88595.
For epoch 30, batch #3732. Training cost is 0.3822263530100157 and training accuraccy is 0.8859333333333334.
For epoch 30, batch #3733. Training cost is 0.3804595610066633 and training accuraccy is 0.8868166666666667.
For epoch 30, batch #3734. Training cost is 0.3796756093412178 and training accuraccy is 0.8878833333333334.
For epoch 30, batch #3735. Training cost is 0.3798354460257199 and training accuraccy is 0.8882666666666666.
For epoch 30, batch #3736. Training cost is 0.3717513915756226 and training accuraccy is 0.8907666666666667.
For epoch 30, batch #3737. Training cost is 0.37296028512681256 and training accuraccy is 0.8907333333333334.
For epoch 30, batch #3738. Training cost is 0.3763783144166961 and training accuraccy is 0.8890333333333333.
For epoch 30, batch #3739. Training cost is 

(0.43601328177847704, 0.8684)